## پروژه نهایی داده کاوی گروه محمدرضا شهبازی 40119973 و یکتا بیات 40116463

#### توجه : برای جلوگیری از گرفتن وقت ، شما میتوانید ابتدا بدون ران گرفتن پروژه خروجی های گرفته شده در سیستم بنده را در خروجی هر بخش مشاهده و بررسی کنید و سپس برای ارزیابی دقیق تر در صورت نیاز پروژه را پس از نصب نیازمندی های پروژه اجرا بفرمایید 

#### در ضمن در صورت بروز هر گونه ابهام یا مشکل در اجرا و ارزیابی پروژه ، بنده در هر ساعتی در تلگرام با آیدی زیر آماده پاسخگویی و رفع ابهام هستم با تشکر از توجه و وقت ارزشمند شما 

https://t.me/Mmreza_81

### نصب نیازمندی‌ها
##### لطفا ابتدا نیازمندی‌های پروژه را در ترمینال نصب کنید. برای این کار از دستور زیر استفاده کنید :

# Install Dependencies

First, install the dependencies required for the project in your terminal. Use the following command:

```bash
pip install -r requirements.txt


pip install -r requirements.txt



# Step 1: Loading and Preprocessing the Data

In this step, the data is loaded from a CSV file and preprocessed. This process includes converting text to lowercase, removing punctuation, and filtering out stopwords.

---

# مرحله اول: بارگذاری و پیش‌پردازش داده‌ها

در این مرحله، داده‌ها از فایل بارگذاری شده و پیش‌پردازش می‌شوند. این فرآیند شامل تبدیل متن به حروف کوچک، حذف نشانه‌گذاری‌ها و توقف‌کلمات است.


In [5]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)

def preprocess_text(text):
    """ Convert text to lowercase, remove punctuation, and remove stopwords """
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  
    stop_words = set(stopwords.words('english'))
    return ' '.join(word for word in text.split() if word not in stop_words)

def load_and_clean_data(file_path):
    """ Load data and convert ratings to sentiment labels """
    df = pd.read_csv(file_path)

    def rating_to_sentiment(rating):
        try:
            numeric_rating = float(re.sub(r'[^\d.]', '', str(rating))) 
            return 1 if numeric_rating > 3 else 0  # Threshold of 3
        except:
            return 0  

    df['sentiment'] = df['rating'].apply(rating_to_sentiment)
    df['processed_review'] = df['review'].apply(preprocess_text)
    return df


# Step 2: Text Vectorization

In this step, we use three different methods for vectorizing the text into numerical representations:
- **TF-IDF**
- **Word2Vec**
- **BERT**

---

# مرحله دوم: بردار سازی متن

در این مرحله، از سه روش بردار سازی مختلف برای تبدیل متن به بردارهای عددی استفاده می‌کنیم:
- **TF-IDF**
- **Word2Vec**
- **BERT**

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer

def vectorize_text(texts, method='tfidf'):
    """ Convert text to numerical representations using TF-IDF, Word2Vec, or BERT """
    if method == 'tfidf':
        vectorizer = TfidfVectorizer(max_features=5000)
        return vectorizer.fit_transform(texts), vectorizer

    elif method == 'word2vec':
        tokenized_texts = [text.split() for text in texts]
        w2v_model = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

        vectors = np.array([
            np.mean([w2v_model.wv[word] for word in words if word in w2v_model.wv] or [np.zeros(100)], axis=0)
            for words in tokenized_texts
        ])
        return vectors, w2v_model

    elif method == 'bert':
        bert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        return bert_model.encode(texts.tolist(), show_progress_bar=True), bert_model

    else:
        raise ValueError("Invalid vectorization method. Please choose 'tfidf', 'word2vec', or 'bert'.")


# Step 3: Training Models with Best Hyperparameters

In this step, we train several models such as **Logistic Regression**, **Random Forest**, and **K-Nearest Neighbors** using grid search to find the best hyperparameters. The models are evaluated based on their **F1 score**, and if the F1 score is below 0.8, the model is retrained.

---

# مرحله سوم: آموزش مدل‌ها با تنظیمات بهترین پارامترها

در این مرحله، چندین مدل مختلف مانند **Logistic Regression**، **Random Forest** و **K-Nearest Neighbors** با استفاده از جستجوی شبکه‌ای (Grid Search) برای تنظیم بهترین پارامترها آموزش داده می‌شوند. مدل‌ها بر اساس امتیاز **F1** ارزیابی می‌شوند و در صورتی که امتیاز F1 به بالای ۰.۸ نرسد، مدل دوباره آموزش داده می‌شود.


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, classification_report

def train_models_with_f1_requirement(X_train, X_test, y_train, y_test, min_f1=0.8):
    """ Train models with hyperparameter tuning and ensure F1 score is above 0.8 """
    models = {
        'logistic_regression': {
            'model': LogisticRegression(),
            'params': {
                'C': [0.1, 1, 10],
                'max_iter': [5000],
                'penalty': ['l1', 'l2'],
                'solver': ['liblinear']
            }
        },
        'random_forest': {
            'model': RandomForestClassifier(),
            'params': {
                'n_estimators': [100, 200],
                'max_depth': [10, 20, None],
                'min_samples_split': [2, 5],
                'min_samples_leaf': [1, 2]
            }
        },
        'knn': {
            'model': KNeighborsClassifier(),
            'params': {
                'n_neighbors': [3, 5, 7],
                'weights': ['uniform', 'distance'],
                'metric': ['euclidean', 'manhattan']
            }
        }
    }

    best_model = None
    best_score = 0
    best_results = None

    for model_name, details in models.items():
        print(f"\nTraining {model_name} model...")

        while True:
            grid_search = GridSearchCV(details['model'], details['params'], cv=5, scoring='f1', n_jobs=-1)
            grid_search.fit(X_train, y_train)

            optimal_model = grid_search.best_estimator_
            y_pred = optimal_model.predict(X_test)
            f1 = f1_score(y_test, y_pred)

            print(f"Model: {model_name} | Best F1: {f1:.4f} | Best Parameters: {grid_search.best_params_}")

            if f1 >= min_f1:
                break  

        if f1 > best_score:
            best_model = model_name
            best_score = f1
            best_results = {
                'model': optimal_model,
                'params': grid_search.best_params_,
                'f1_score': f1,
                'classification_report': classification_report(y_test, y_pred)
            }

    return best_model, best_results


# Step 4: Generate HTML Report from Best Model Results

In this step, the results of the best model, including the F1 score and classification report, are converted into an HTML format for easy viewing.

---

# مرحله چهارم: ایجاد گزارش HTML از نتایج بهترین مدل

در این مرحله، نتایج بهترین مدل از جمله امتیاز F1 و گزارش طبقه‌بندی به فرمت HTML تبدیل می‌شوند تا به راحتی بتوانیم نتایج را مشاهده کنیم.
 به راحتی میتوانید فایل اچ تی ام ال را خارج از کامپایلر یا وی اس کد و ... در مرورگر خود باز کنید و مشاهده کنید. همچنین جداگانه میتوانید در خروجی سکشن پایین پارامتر ها و دقت هر مدل و متود را مشاهده کنید و فایل اچ تی ام ال صرفا جنبه زیبایی دارد. با تشکر از توجه شما

In [9]:
data = load_and_clean_data('train_sentiment.csv')

vector_method = 'tfidf'
X_vectorized, vectorizer = vectorize_text(data['processed_review'], method=vector_method)

X_train, X_test, y_train, y_test = train_test_split(X_vectorized, data['sentiment'], test_size=0.3, random_state=42)

best_model_name, best_model_info = train_models_with_f1_requirement(X_train, X_test, y_train, y_test)

print(f"\n🔥 Best Model: {best_model_name}")
print(f"🔹 F1 Score: {best_model_info['f1_score']:.4f}")
print("🔹 Classification Report:\n", best_model_info['classification_report'])

NameError: name 'train_test_split' is not defined

In [16]:
html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Best Model Results</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #ffffff;
            color: #333;
            margin: 0;
            padding: 0;
        }}
        .container {{
            width: 80%;
            margin: 50px auto;
            padding: 20px;
            border-radius: 10px;
            background-color: #f4f4f4;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.1);
        }}
        h1 {{
            text-align: center;
            color: #2c3e50;
        }}
        .result-table {{
            width: 100%;
            border-collapse: collapse;
            margin: 30px 0;
        }}
        .result-table th, .result-table td {{
            padding: 12px;
            text-align: left;
            border-bottom: 1px solid #ddd;
        }}
        .result-table th {{
            background-color: #3498db;
            color: white;
        }}
        .result-table tr:hover {{
            background-color: #f2f2f2;
        }}
        .classification-report {{
            margin-top: 30px;
            padding: 15px;
            border: 1px solid #ddd;
            background-color: #eaf1f9;
            border-radius: 8px;
        }}
        .classification-report h2 {{
            text-align: center;
            color: #3498db;
        }}
        .classification-report pre {{
            background-color: #fff;
            padding: 20px;
            border-radius: 5px;
            white-space: pre-wrap;
            word-wrap: break-word;
            color: #2c3e50;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1>Best Performing Model</h1>
        <table class="result-table">
            <tr>
                <th>Metric</th>
                <th>Value</th>
            </tr>
            <tr>
                <td>🔥 Best Model</td>
                <td>{best_model_name}</td>
            </tr>
            <tr>
                <td>✅ F1 Score</td>
                <td>{best_model_info['f1_score']:.4f}</td>
            </tr>
        </table>

        <div class="classification-report">
            <h2>📊 Classification Report:</h2>
            <pre>{best_model_info['classification_report']}</pre>
        </div>
    </div>
</body>
</html>
"""
with open("model_results.html", "w") as file:
    file.write(html_content)

print("HTML report saved as 'model_results.html'. Open this file to view the results.")

HTML report saved as 'model_results.html'. Open this file to view the results.
